In [2]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('driftpy/src/')

from driftpy.types import User
from driftpy.constants.config import configs
from anchorpy import Provider
import json 
from anchorpy import Wallet
from solana.rpc.async_api import AsyncClient
from driftpy.clearing_house import ClearingHouse
from driftpy.accounts import *
from solana.publickey import PublicKey
from solana.keypair import Keypair
import pathlib 
from tqdm.notebook import tqdm 
import shutil
from anchorpy import Instruction
import base64
from subprocess import Popen
import os 
import time
import signal
from driftpy.admin import Admin
from helpers import *

script_file = 'start_local.sh'
os.system(f'cat {script_file}')
print()

validator = LocalValidator(script_file)
validator.start()

config = configs['devnet']
url = 'http://127.0.0.1:8899'
connection = AsyncClient(url)

chs, state_ch = await load_local_users(config, connection)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
#!/bin/bash
./solana/target/debug/solana-test-validator --account-dir accounts/Oracles --account-dir accounts/Bank --account-dir accounts/User --account-dir accounts/State --account-dir accounts/UserStats --account-dir accounts/Market --bpf-program 65sz7dRiWDRPZjiRxcTxPM7AE6VK4Nag9HEK6oBJXhJn accounts/65sz7dRiWDRPZjiRxcTxPM7AE6VK4Nag9HEK6oBJXhJn.so --resetfound admin...


In [5]:
await state_ch.update_auction_duration(0, 0)
await state_ch.update_max_base_asset_amount_ratio(1, 0)

'5TQ377yh7wBWuD2nYHD2qVmbrDUbrV41egG9xHbiUhWjEuUfnH7FUzc1dXZesDK7PE4LnYygCtuGBepvQ9M1JG29'

In [8]:
# note: sometimes need to run this twice for lps (remove on first loop then close on second)
close_out = False
net_baa = 0 
sigs = []
for ch in tqdm(chs):
    position = await ch.get_user_position(0)

    if position.lp_shares > 0:
        if close_out:
            print('removing...', position.lp_shares)
            sig = await ch.remove_liquidity(position.lp_shares, 0)
            sigs.append(sig)

    if position.base_asset_amount != 0:
        net_baa += position.base_asset_amount
        if close_out:
            print('closing...', position.base_asset_amount / 1e13)
            sig = await ch.close_position(0)
            sigs.append(sig)
    

  0%|          | 0/82 [00:00<?, ?it/s]

In [10]:
if close_out:
    # wait for txs to confirm
    while True:
        resp = await connection.get_transaction(sigs[-1])
        if resp['result'] is not None: 
            break 

market = await get_market_account(state_ch.program, 0)
net_baa, market.amm.net_base_asset_amount

(-382198840000000, 17179431531215116)

In [11]:
# shutdown validator
validator.stop()

ProcessLookupError: [Errno 3] No such process